## Method 1

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

In [2]:
ds = tfds.load('squad', split='train', shuffle_files=True)
assert isinstance(ds, tf.data.Dataset)

In [10]:
for i in ds:
    pass

## Method 2 (Used)

In [4]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [1]:
import json
import pandas as pd

In [2]:
f = open('train-v2.0.json')
json_file = json.load(f)

In [ ]:
title = []
context = []
question = []
answer = []
for i in json_file['data']:
    title.append(i['title'])
    tempq = []
    tempa = []
    tempc = ''
    for j in i['paragraphs']:
        tempc += j['context'] + ' '
        for k in j['qas']:
            if len(k['answers']) == 0:
                continue
            tempq.append(k['question'])
            tempa.append(k['answers'][0])
    context.append(tempc)
    question.append(tempq)
    answer.append(tempa)

In [10]:
df = pd.DataFrame({'title':title, 'context':context, 'question':question, 'answer':answer})

In [15]:
df.to_pickle('data.df')

In [16]:
df = pd.read_pickle('data.df')

In [17]:
for i in range(len(df)):
    if len(df['question'][i])!=len(df['answer'][i]):
        print('lol')

## Prepare question generation dataset from SQuaD

In [68]:
from nltk.tokenize import sent_tokenize
import json
import pandas as pd

In [141]:
f = open('train-v2.0.json')
json_file = json.load(f)

In [143]:
qg_dataset = []
num_qa = 0
num_exact = 0 

for i in json_file['data']:
    title = i['title']
    context = []
    question = []
    answer = []
    for j in i['paragraphs']:
        for k in j['qas']:
            if len(k['answers']) == 0:
                continue
            
            num_qa+=1
            qg_row = {}

            qg_row['question'] = k['question']
            sentences = sent_tokenize(j['context'])
            found = False
            for ans in k['answers']:
                for s in range(len(sentences)):
                    if ans['text'] in sentences[s] and not found:
                        num_exact+=1
                        sent = sentences[s]
                        found = True
                        break
                    
            qg_row['sentence'] = sent
            qg_row['answer'] = ans['text']
            qg_row['context'] = j['context']
            qg_row['exact'] = found
            qg_dataset.append(qg_row)
            
print(f"Exact span found for {round(num_exact/num_qa, 3)} of data")

Exact span found for 0.998 of data


In [144]:
qg_df = pd.DataFrame(qg_dataset)

In [145]:
qg_df.iloc[0].to_dict()

{'question': 'When did Beyonce start becoming popular?',
 'sentence': "Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child.",
 'answer': 'in the late 1990s',
 'context': 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby 

In [146]:
# failed to identify sentence containing answer
qg_df[qg_df['exact']==False]

,question,sentence,answer,context,exact
895,What was the first of Chopin's works to gain i...,The success of this concert led to an invitati...,Rondo Op. 1,From September 1823 to 1826 Chopin attended th...,False
900,What is the title of his first commercially su...,The success of this concert led to an invitati...,Rondo Op. 1.,From September 1823 to 1826 Chopin attended th...,False
928,Which of Chopin's works was unknowingly dedica...,He was also attracted to the singing student K...,Piano Concerto No. 1 (in E minor),Four boarders at his parents' apartments becam...,False
933,What musical piece had a Larghetto dedicated t...,Four boarders at his parents' apartments becam...,Piano Concerto No. 1 (in E minor),Four boarders at his parents' apartments becam...,False
938,What piece did Frédéric create specifically fo...,"On an 1829 return trip to Berlin, he was a gue...",Introduction and Polonaise brillante in C majo...,"In September 1828 Chopin, while still a studen...",False
...,...,...,...,...,...
85043,Which president did Notre Dame have in 1947?,"He wrote, ""Notre Dame football is a spiritual ...","Rev. John J. Cavanaugh, C.S.C.","The Rev. John J. Cavanaugh, C.S.C. served as p...",False
85073,Which person oversaw the creation of a science...,"The Grotto of Our Lady of Lourdes, which was b...",Fr. Zahm,A Science Hall was built in 1883 under the dir...,False
85249,"Who is responsible for writing the music for ""...","Michael J. Shea, a 1904 graduate, wrote the mu...",Rev. Michael J. Shea,"The ""Notre Dame Victory March"" is the fight so...",False
85259,Who is the current president of Notre Dame?,A notable alumnus of the College of Science is...,Rev. John Jenkins,Notre Dame alumni work in various fields. Alum...,False


In [147]:
qg_df[qg_df['exact']==True].to_csv('qg_train.csv', index=False, encoding='utf-8')

# Analyse question generation dataset

In [1]:
import pandas as pd
from nltk import word_tokenize
import matplotlib.pyplot as plt

In [148]:
train_df = pd.read_csv("qg_train.csv")
dev_df = pd.read_csv("qg_dev.csv")

In [149]:
print(f"# of samples: \nTrain = {len(train_df)}\nDev = {len(dev_df)}")

# of samples: 
Train = 86607
Dev = 5923


In [90]:
# see what is distribution of sentence, answer and question lengths
for col in ['sentence', 'question', 'answer']:
    print(col)
    len_col = col+"_length"
    train_df[len_col] = train_df[col].apply(lambda x: len(word_tokenize(str(x))))

sentence
question
answer


In [91]:
train_df['input_length'] = train_df['sentence_length']+train_df['answer_length']

In [92]:
train_df.describe()

,sentence_length,question_length,answer_length,input_length
count,86607.000000,86607.000000,86607.000000,86607.000000
mean,32.163878,11.293106,3.355595,35.519473
std,16.515101,3.718376,3.742623,17.144897
min,2.000000,1.000000,1.000000,3.000000
25%,21.000000,9.000000,1.000000,24.000000
50%,29.000000,11.000000,2.000000,32.000000
75%,39.000000,13.000000,4.000000,43.000000
max,382.000000,60.000000,46.000000,385.000000


In [93]:
train_df.quantile(0.9)

exact               1.0
sentence_length    52.0
question_length    16.0
answer_length       7.0
input_length       56.0
Name: 0.9, dtype: float64

In [98]:
train_df.quantile(0.1)

exact               1.0
sentence_length    15.0
question_length     7.0
answer_length       1.0
input_length       18.0
question_word       1.0
Name: 0.1, dtype: float64

In [105]:
train_df[train_df['question_length']<3]

,question,sentence,answer,context,exact,sentence_length,question_length,answer_length,input_length,question_word
35704,dd,The religious sphere expanded to include new g...,Buddhism,The Hellenistic period covers the period of an...,True,28,1,1,29,False
35705,d,The Hellenistic period covers the period of an...,the Gre,The Hellenistic period covers the period of an...,True,54,1,2,56,False
53361,What means,"Advaita literally means ""not two, sole, unity"".",Advaita,"Advaita literally means ""not two, sole, unity""...",True,12,2,1,13,True


In [94]:
question_words = ["how", "what", "where", "when", "who", "which", "why"]
train_df['question_word'] = train_df['question'].apply(lambda x: any(ele in x.lower() for ele in question_words) or x.lower().split()[0] in ['is', 'are', 'was', 'were', 'does', 'do', 'did', 'has', 'had', 'have', 'can', 'could', 'will', 'would', 'should', 'name', 'list', 'describe', 'give', 'if'])

In [96]:
train_df[train_df['question_word']==False]

,question,sentence,answer,context,exact,sentence_length,question_length,answer_length,input_length,question_word
230,Hoe did everyone learn that Beyonce performed ...,"In 2011, documents obtained by WikiLeaks revea...",documents obtained by WikiLeaks,"In 2011, documents obtained by WikiLeaks revea...",True,26,10,4,30,False
477,Hoe does she describe Oprah Winfrey?,Beyoncé has stated that she is personally insp...,a strong woman,Beyoncé has stated that she is personally insp...,True,43,7,3,46,False
486,Beyonce had singers in the background known by...,"Her background singers, The Mamas, consist of ...",The Mamas,"In 2006, Beyoncé introduced her all-female tou...",True,19,12,2,21,False
693,Beyonce's grandma's name was?,The concept is inspired by three generations o...,Agnèz Deréon,Beyoncé and her mother introduced House of Der...,True,29,7,2,31,False
2531,Whic organization is responsible for filtering...,"In India, it was reported that the Indian Cent...",Indian Central Board of Film Certification,"In India, it was reported that the Indian Cent...",True,31,11,6,37,False
...,...,...,...,...,...,...,...,...,...,...
83448,IndyMac offered this type of questionable loan...,"As an Alt-A lender, IndyMac’s business model w...",risky,IndyMac often made loans without verification ...,True,46,10,1,47,False
83449,IndyMac resisted efforts by regulators to tigh...,IndyMac resisted efforts to regulate its invol...,issuing criteria,IndyMac often made loans without verification ...,True,32,13,2,34,False
83516,Arab countries entered the financial crisis in...,This gives them a significant cushion against ...,global downturn,The World Bank reported in February 2009 that ...,True,11,17,2,13,False
86119,In Victorian times a popular social activity was?,Unarmed fox hunting on horseback with hounds i...,fox hunting,Unarmed fox hunting on horseback with hounds i...,True,32,9,2,34,False


In [129]:
train_df[(train_df['answer_length']>train_df['answer_length'].quantile(0.9))]

,question,sentence,answer,context,exact,sentence_length,question_length,answer_length,input_length,question_word
198,"Where did Beyoncé perform on January 20, 2009?","On January 20, 2009, Beyoncé performed James' ...",the First Couple's first inaugural ball.,"Beyoncé further expanded her acting career, st...",True,23,10,8,31,True
443,Beyonce received the Pop Songwriter of the Yea...,"In 2001, she became the first African-American...","American Society of Composers, Authors, and Pu...","In 2001, she became the first African-American...",True,37,13,12,49,True
451,Who gave Beyoncé the Pop Songwriter of the Yea...,"In 2001, she became the first African-American...","the American Society of Composers, Authors, an...","In 2001, she became the first African-American...",True,37,13,14,51,True
487,The Mamas members included which 3 musicians?,"Her background singers, The Mamas, consist of ...","Montina Cooper-Donnell, Crystal Collins and Ti...","In 2006, Beyoncé introduced her all-female tou...",True,19,8,9,28,True
636,What Beyoncé song was song of the year on 2010?,"""Single Ladies (Put a Ring on It)"" won Song of...","""Single Ladies (Put a Ring on It)""","Beyoncé has won 20 Grammy Awards, both as a so...",True,39,11,11,50,True
...,...,...,...,...,...,...,...,...,...,...
86288,What is the requirement of the act?,"On 16 March 1934, President Franklin D. Roosev...",annual stamp purchase by all hunters over the ...,"On 16 March 1934, President Franklin D. Roosev...",True,32,8,11,43,True
86307,What did the large antelope species once inhabit?,"The Arabian oryx, a species of large antelope,...",much of the desert areas of the Middle East,"The Arabian oryx, a species of large antelope,...",True,22,9,9,31,True
86377,"In the waning years of the Malla dynasty, what...","During the later part of the Malla era, Kathma...","Kantipur, Lalitpur, Bhaktapur, and Kirtipur","During the later part of the Malla era, Kathma...",True,25,18,8,33,True
86378,What cultures influenced Nepal in the later Ma...,"Evidence of an influx of ideas from India, Tib...","India, Tibet, China, Persia, and Europe","During the later part of the Malla era, Kathma...",True,35,10,10,45,True


In [110]:
train_df[train_df['sentence_length']<7]

,question,sentence,answer,context,exact,sentence_length,question_length,answer_length,input_length,question_word
614,Who was the first female to achieve the Intern...,Beyoncé has received numerous awards.,Beyoncé,Beyoncé has received numerous awards. As a sol...,True,6,17,1,7,True
946,On what date did Frédéric give his first perfo...,21 on 17 March 1830.,17 March 1830,"Back in Warsaw that year, Chopin heard Niccolò...",True,6,23,3,9,True
2307,How does White deal with his condition?,White then commits suicide.,commits suicide,"Bond travels to Austria to find White, who is ...",True,5,8,2,7,True
2437,How long did it take to shoot Spectre?,Filming took 128 days.,128 days,"After wrapping up in England, production trave...",True,5,9,2,7,True
2440,How many day did filming take,Filming took 128 days.,128 days.,"After wrapping up in England, production trave...",True,5,6,3,8,True
...,...,...,...,...,...,...,...,...,...,...
82826,What did Eisenhower do when he was encouraged ...,"Nevertheless, he refused.",he refused,This prevented Eisenhower from openly condemni...,True,5,15,2,7,True
83283,What encouraged borrowing from 2000 to 2003?,Lower interest rates encouraged borrowing.,Lower interest rates,Lower interest rates encouraged borrowing. Fro...,True,6,8,3,9,True
84587,How many rooms does the largest hotel in St. B...,The largest has 58 rooms.,58,"St. Barthélemy has about 25 hotels, most of th...",True,6,12,1,7,True
84933,What person was the Director of the Science Mu...,"Father Joseph Carrier, C.S.C.","Father Joseph Carrier, C.S.C.","Father Joseph Carrier, C.S.C. was Director of ...",True,6,18,6,12,True


In [111]:
e = 3
predicate = (train_df['sentence_length']<7) \
            & (train_df['sentence_length']>=train_df['answer_length']) \
            & (train_df['sentence_length']<=train_df['answer_length']+e)
   
train_df[predicate].sort_values(by='sentence_length')

,question,sentence,answer,context,exact,sentence_length,question_length,answer_length,input_length,question_word
32617,Who reported that Nintendo sold 7 million NES ...,Compute!,Compute!,"By 1988, industry observers stated that the NE...",True,2,10,2,4,True
39466,Until what year did other models use the origi...,1955.,1955,The Weston Cadet (model 852 introduced in 1949...,True,2,12,1,3,True
35400,The Propaganda Due Lodge was also know as what?,P2).,P2,"In Italy, Freemasonry has become linked to a s...",True,3,10,1,4,True
8139,When did the Andronovo culture exist?,2000 BC.,2000 BC,All Iranian languages are descended from a com...,True,3,7,2,5,True
25034,Which section indicates the methods in which t...,§ 5112.,5112,The Constitution of the United States of Ameri...,True,3,13,1,4,True
77329,In what year did Sviatoslav take over as leader?,963).,963,Following the death of Grand Prince Igor in 94...,True,3,10,1,4,True
23072,"when was mr. fingers' ""can you feel it?"" relea...",(1986).,1986,Deep house's origins can be traced to Chicago ...,True,4,14,1,5,True
30938,What is BSAR-1 a strain of?,A Sphingomonas sp.,Sphingomonas sp.,"In nature, uranium(VI) forms highly soluble ca...",True,4,7,3,7,True
35931,What was the Jewish elite class called?,the Tobiads).,the Tobiads,Between 301 and 219 BCE the Ptolemies ruled Ju...,True,4,8,2,6,True
23908,What type of weather is a rarity in Houston?,Snowfall is rare.,Snowfall,Houston has mild winters in contrast to most a...,True,4,10,1,5,True


In [11]:
predicate = (train_df['sentence_length']==train_df['answer_length']) 
            
train_df[predicate].sort_values(by='sentence_length')

,question,sentence,answer,context,exact,sentence_length,question_length,answer_length,input_length
32617,Who reported that Nintendo sold 7 million NES ...,Compute!,Compute!,"By 1988, industry observers stated that the NE...",True,2,10,2,4
84933,What person was the Director of the Science Mu...,"Father Joseph Carrier, C.S.C.","Father Joseph Carrier, C.S.C.","Father Joseph Carrier, C.S.C. was Director of ...",True,6,18,6,12
27047,Where on Earth is biodiversity the most abundn...,It is richest in the tropics.,It is richest in the tropics.,"Biodiversity, a contraction of ""biological div...",True,7,9,7,14
37112,What happens to the energy that an incandescen...,The remaining energy is converted into heat.,The remaining energy is converted into heat.,Incandescent bulbs are much less efficient tha...,True,8,15,8,16
46938,Does Raleigh have alternate high schools?,Raleigh also has two alternative high schools.,Raleigh also has two alternative high schools.,Public schools in Raleigh are operated by the ...,True,8,7,8,16
...,...,...,...,...,...,...,...,...,...
69115,What is the states history?,The state outside the Gadsden Purchase of 1853...,The state outside the Gadsden Purchase of 1853...,The state (like its southwestern neighbors) ha...,True,29,6,29,58
39215,How could a wrestler win in the past?,"Historically, matches were wrestled to 3 falls...","Historically, matches were wrestled to 3 falls...",Most wrestling matches last for a set number o...,True,30,9,30,60
39212,What kind of conflict can arise during a show?,"Often, individual matches will be part of a lo...","Often, individual matches will be part of a lo...",Professional wrestling shows can be considered...,True,30,10,30,60
4840,What type of community was Siddhārtha Gautama ...,"It was either a small republic, in which case ...","It was either a small republic, in which case ...",The evidence of the early texts suggests that ...,True,30,10,30,60


In [150]:
df_map = {"qg_train":train_df, "qg_dev":dev_df}
for name in df_map:
    print(name)
    
    df = df_map[name]
    print(len(df))
    
    if set(['sentence_length', 'answer_length', 'question_length']) - set(df.columns):
        
        for col in ['sentence', 'answer', 'question']:
            print(col)
            len_col = col+"_length"
            df[len_col] = df[col].apply(lambda x: len(word_tokenize(str(x))))
        
    #throw out samples that have short questions, long answers, sentences not long enough for answers
    e = 3
    predicate = ((df['question_length']<4) \
                | (df['answer_length']>df['answer_length'].quantile(0.9))\
                | (df['sentence_length']<7) \
                    & (df['sentence_length']>=df['answer_length']) \
                    & (df['sentence_length']<=df['answer_length']+e))
    
    drop_idx = df[predicate].index
    
    df.drop(index=drop_idx, inplace=True)
    print(len(df))
    df[['question', 'answer', 'sentence', 'context']].to_csv(name+'.csv', index=False)

qg_train
86607
sentence
answer
question
78610
qg_dev
5923
sentence
answer
question
5409


## Check different questions for the same sentence

In [151]:
import pandas as pd

In [154]:
dev_df = pd.read_csv('qg_dev.csv')

In [166]:
gb = dev_df.groupby(by='sentence')
agg = gb.count()

In [170]:
drop_idx = agg[agg['question']==1].index
agg.drop(index=drop_idx, inplace=True)

In [171]:
for gg, dd in gb:
    if gg in agg.index:
        print(gg)
        print(dd['question'])

"Markets, by themselves, even when they are stable, often lead to high levels of inequality, outcomes that are widely viewed as unfair."
2986    How is income inequality generally viewed by w...
2991    What type of outcomes can even stable markets ...
Name: question, dtype: object
"The Islamic State", formerly known as the "Islamic State of Iraq and the Levant" and before that as the "Islamic State of Iraq", (and called the acronym Daesh by its many detractors), is a Wahhabi/Salafi jihadist extremist militant group which is led by and mainly composed of Sunni Arabs from Iraq and Syria.
4643    What type of group is The Islamic State?
4644                Who leads The Islamic State?
Name: question, dtype: object
'"[attribution needed] Because of the resources made available by imperialism, the world's economy grew significantly and became much more interconnected in the decades before World War I, making the many imperial powers rich and prosperous.
4747    What grew on a global scale 